<img src="../assets/logo-bdc.png" align="right" width="64"/>

# <span style="color: #336699">Earth Observation Data Cubes for Brazil: Requirements, Methodology and Products</span>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 90%;">
    Rolf E. O. Simões <sup><a href="mailto:rolf.simoes@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0003-0953-4132"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Alber H. Sanchez <sup><a href="mailto:alber.ipia@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0001-7966-2880"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Felipe M. Carlos <sup><a href="mailto:felipe.carlos@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0002-3334-4315"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Leonardo S. Vieira <sup><a href="mailto:leonardo.vieira@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0002-3397-6232"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>,<br/>
    Karine R. Ferreira <sup><a href="mailto:karine.ferreira@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0003-2656-5504"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Lubia Vinhas <sup><a href="mailto:lubia.vinhas@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0003-1104-3607"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Gilberto R. Queiroz<sup>* <a href="mailto:gilberto.queiroz@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0001-7534-0219"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Earth Observation and Geoinformatics Division, National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    <sup>*</sup> Author to whom correspondence should be addressed.
    <br/><br/>
    February 24, 2021
</div>

<br/>

<div style="text-align: justify;  margin-left: 10%; margin-right: 10%;">
<b>Abstract.</b> This Jupyter Notebook compendium contains useful information for the creation of land use and land cover (LULC) maps using Earth observations data cubes and machine learning (ML) techniques. The code is based on the research pipeline described in the paper <em>Earth Observation Data Cubes for Brazil: Requirements, Methodology and Products</em>. All the datasets required to the reproducibility of the work is also available. 
</div>    

<br/>
<div style="text-align: justify;  margin-left: 15%; margin-right: 15%;font-size: 75%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>This Jupyter Notebook is supplement to the <a href="https://www.mdpi.com/2072-4292/12/24/4033/htm#sec5-remotesensing-12-04033" target="_blank">Section 5</a> of the following paper:</b>
    <div style="margin-left: 10px; margin-right: 10px">
    Ferreira, K.R.; Queiroz, G.R.; Vinhas, L.; Marujo, R.F.B.; Simoes, R.E.O.; Picoli, M.C.A.; Camara, G.; Cartaxo, R.; Gomes, V.C.F.; Santos, L.A.; Sanchez, A.H.; Arcanjo, J.S.; Fronza, J.G.; Noronha, C.A.; Costa, R.W.; Zaglia, M.C.; Zioti, F.; Korting, T.S.; Soares, A.R.; Chaves, M.E.D.; Fonseca, L.M.G. 2020. Earth Observation Data Cubes for Brazil: Requirements, Methodology and Products. Remote Sens. 12, no. 24: 4033. DOI: <a href="https://doi.org/10.3390/rs12244033" target="_blank">10.3390/rs12244033</a>.
    </div>
</div>

# <span style="color: #336699">Validation results for Landsat-8/OLI classification</span>
<hr style="border:1px solid #0077b9;">

This document presents the process for the validation of the results generated in the classification made using the Landsat-8/OLI data cube. In this example, the validation process was done using the [SITS R package](https://github.com/e-sensing/sits).

In [ ]:
library(sf)
library(sits)
library(caret)
library(dplyr)
library(raster)
library(magrittr)

## <span style="color: #336699">Load data</span>

To perform the validation, it is initially necessary to load the classification results and the samples used as a reference.

In [ ]:
#
# Defining the base directories
#
data_input  <- fs::path("../data/raw_data")
data_output <- fs::path("../data/derived_data")

#
# Load the validation samples
#
validation_samples <- sf::read_sf(data_input / "samples" / "validation" / "data.shp")

#
# Load the LULC classification map (Generated in previous step)
#
lulc_classification_map <- terra::rast(
    fs::dir_ls(
        data_output / "classification" / "LC8_30_16D_STK-1", regexp = "LANDSAT-8_OLI_merged_*"
    )
)

## <span style="color: #336699">Preparing the validation samples</span>

The LULC samples used as a reference in the comparisons have class names that prevent direct comparison with the data generated in the time series classification. In this step, we will replace these values to be compared. To do this, the values in the variable `validation_samples`, which we loaded above, will be changed according to the following rule:

- `Natural` -> `1`;
- `Anthropic` -> `2`.

In [ ]:
#
# Transforming the reference date.
#
reference_data <- dplyr::mutate(
    .data     = validation_samples, 
    reference = factor(ifelse(reference == "Anthropic", 2, 1))
)

head( reference_data )

In [ ]:
#
# Generating the validation sample data.
#
predicted_values <- 
    reference_data %>%
        sf::st_transform(sf::st_crs(terra::crs(lulc_classification_map))) %>% 
        dplyr::select(geometry) %>%
        sfheaders::sf_to_df() %>%
        dplyr::rename(long=x, lat=y) %>%
        dplyr::select(c("long", "lat"))

head( predicted_values )

## <span style="color: #336699">Extract predicted values</span>

With the rasters and reference samples loaded, it will now be necessary to extract the values corresponding to each of the validation samples' locations from the classified raster. This extraction allows the comparison of the predicted and reference values.

> For extraction at each location, the `extract` function of the [raster package](https://cran.r-project.org/web/packages/raster/raster.pdf) is used.


In [ ]:
predicted_values <- terra::extract(
                      x = lulc_classification_map, 
                      y = predicted_values
                    )

head( predicted_values )

## <span style="color: #336699">Transform extracted values</span>

As described in the article, the classified elements were converted so that the evaluation process could be carried out. Here, the `Agriculture` and `Pasture` elements are unified to represent the `Anthropic` class of PRODES. The `Natural Vegetation` class is not changed for comparison.

> When the classification process is carried out, there is no way to specify the values assigned to each of the classes being identified. Thus, it may be necessary to check the classification values and their corresponding elements so that the values are used for comparison. 


In [ ]:
# Natural Vegetation (2)
predicted_values[predicted_values == 2] <- 5

# Pasture (3) and Agriculture (1) to PRODES Anthropic class
predicted_values[predicted_values == 1] <- 2
predicted_values[predicted_values == 3] <- 2

# Natural Vegetation (1)
predicted_values[predicted_values == 5] <- 1

## <span style="color: #336699">Evaluating</span>

In [ ]:
caret::confusionMatrix(
  data      = as.factor(predicted_values$`LANDSAT-8_OLI_merged_2018-08-29_2019-08-29_class_v1`),
  reference = as.factor(reference_data$reference)
)